In [0]:
# Importaciones necesarias
import json
from pyspark.sql import functions as F
from pyspark.sql.functions import col, current_timestamp, round, count, when, lit
from pyspark.sql.types import IntegerType, DoubleType, StructType, StructField, BooleanType, StringType, TimestampType

In [0]:
# 1. Cargar la tabla Bronze
df_bronze = spark.table("workspace.credit_risk.bronze_credit_risk")
#print(f"Total de registros cargados: {df_bronze.count():,}")

In [0]:
df_bronze = (
    df_bronze
    .select(
        F.col("edad").cast("int"),
        F.col("ingreso_anual").cast("double"),
        F.col("tipo_vivienda").alias("tipo_vivienda"),
        F.col("anios_empleo").cast("double"),
        F.col("proposito"),
        F.col("calificacion"),
        F.col("monto").cast("double"),
        F.col("tasa_interes").cast("double"),
        F.col("estado_pago").cast("int"),
        F.col("pct_ingreso").cast("double"),
        F.col("historial_default"),
        F.col("anios_hist_credito").cast("double"),
#        F.col("fecha_ingesta")
    )
)

In [0]:
# Inicializar la lista de reglas
reglas = []

In [0]:
# 2. DEFINICIÓN Y EJECUCIÓN DE REGLAS CRÍTICAS

# 1: No Nulos en Estado pago
n_null_estado_pago = df_bronze.filter((col("estado_pago").isNull())).count()

reglas.append({
    "tabla": "bronze_credit_risk",
    "columna": "estado_pago",
    "regla": "no_null_o_vacio",
    "cumple": n_null_estado_pago == 0,
    "detalle": f"Registros nulos/vacíos: {n_null_estado_pago:,}",
    "nivel": "CRITICO" # Añadimos un nivel para distinguir
})

# 2: No Nulos en monto
n_null_monto = df_bronze.filter((col("monto").isNull())).count()

reglas.append({
    "tabla": "bronze_credit_risk",
    "columna": "monto",
    "regla": "no_null_o_vacio",
    "cumple": n_null_monto == 0,
    "detalle": f"Registros nulos/vacíos: {n_null_monto:,}",
    "nivel": "CRITICO" # Añadimos un nivel para distinguir
})

# 3. NUEVA REGLA DE DOMINIO: Tasa de Interés No Negativa.
tasas_negativas = df_bronze.filter((col("tasa_interes")< 0)).count()

# Esta es una regla de negocio CRÍTICA (una tasa negativa es un error de dato)
regla_cumplida_negativa = tasas_negativas == 0

reglas.append({
    "tabla": "bronze_credit_risk",
    "columna": "tasa_interes",
    "regla": "valor_no_negativo",
    "cumple": regla_cumplida_negativa,
    "detalle": f"Registros con tasa de interés negativa: {tasas_negativas:,}",
    "nivel": "CRITICO"
})

# 4: No nulos en calificacion (segmentación clave)
n_null_calificacion = df_bronze.filter(col("calificacion").isNull()).count()
reglas.append({
    "tabla": "bronze_credit_risk",
    "columna": "calificacion",
    "regla": "no_nulos",
    "cumple": n_null_calificacion == 0,
    "detalle": f"Nulos: {n_null_calificacion:,}",
    "nivel": "CRITICO",
})

# 5. Regla Coherencia monto vs ingreso
# (Préstamo no debe exceder ingreso anual en más de 2x)
total_registros = df_bronze.count()
monto_excesivo = df_bronze.filter(col("monto") > col("ingreso_anual") * 2).count()
pct_monto_excesivo = (monto_excesivo / total_registros * 100) if total_registros > 0 else 0
reglas.append({
    "tabla": "bronze_credit_risk",
    "columna": "monto/ingreso_anual",
    "regla": "monto_max_2x_ingreso",
    "cumple": pct_monto_excesivo < 5,  # Toleramos hasta 5%
    "detalle": f"Préstamos > 2x ingreso: {monto_excesivo:,} ({pct_monto_excesivo:.2f}%)",
    "nivel": "MEDIO",
})

# 6. No nulos en tasa_interes (variable clave de pricing)
n_null_tasa = df_bronze.filter(col("tasa_interes").isNull()).count()
pct_null_tasa = (n_null_tasa / total_registros * 100) if total_registros > 0 else 0
reglas.append({
    "tabla": "bronze_credit_risk",
    "columna": "tasa_interes",
    "regla": "nulos_bajo_umbral",
    "cumple": pct_null_tasa < 5,  # Toleramos hasta 10%
    "detalle": f"Nulos: {n_null_tasa:,} ({pct_null_tasa:.2f}%)",
    "nivel": "CRITICO",
})



In [0]:
# 3. CREAR TABLA DE LOG Y ESCRIBIR RESULTADOS (Adaptado de tus imágenes)

# Definir el esquema para el log (Mejor práctica)
schema_log = StructType([
    StructField("tabla", StringType(), False),
    StructField("columna", StringType(), False),
    StructField("regla", StringType(), False),
    StructField("cumple", BooleanType(), False),
    StructField("detalle", StringType(), True),
    StructField("nivel", StringType(), True)
])

In [0]:
# Crear DataFrame a partir de la lista de diccionarios
df_reglas = spark.createDataFrame(reglas, schema_log)
df_reglas = df_reglas.withColumn("fecha_validacion", current_timestamp())

# Crear la tabla de LOG si no existe (Usando tu comando SQL)
spark.sql("""
CREATE TABLE IF NOT EXISTS workspace.credit_risk.log_reglas_calidad_datos
(
  id BIGINT GENERATED ALWAYS AS IDENTITY,
  tabla STRING,
  columna STRING,
  regla STRING,
  cumple BOOLEAN,
  detalle STRING,
  nivel STRING,
  fecha_validacion TIMESTAMP
)
USING DELTA
""")

In [0]:
# Escribir el log (en modo append)
df_reglas.write.format("delta").mode("append").saveAsTable("workspace.credit_risk.log_reglas_calidad_datos")

In [0]:
# 4. CONTROL DEL WORKFLOW (dbutils.jobs.taskValues.set)

# Filtrar las fallas CRICTICAS (donde 'cumple' es Falso)
fallas_criticas = [regla for regla in reglas if not regla["cumple"] and regla["nivel"] == "CRITICO"]

if fallas_criticas:
    # Si hay fallas críticas, se establece el estado de FALLA CRITICA
    dbutils.jobs.taskValues.set(key="estado", value="Falla Critica")
    # Opcional: Establecer el detalle de la falla (útil para la notificación)
    dbutils.jobs.taskValues.set(key="detalle", value=json.dumps(fallas_criticas))
    print(" FALLA CRÍTICA DETECTADA. Deteniendo el workflow.")
else:
    # Si no hay fallas críticas, se establece el estado de VALIDACION EXITOSA
    dbutils.jobs.taskValues.set(key="estado", value="Validacion Exitosa")
    print(" VALIDACIÓN EXITOSA. Continuando a la Capa Silver.")